In [13]:
import pandas as pd
import os

output_domain_size_dict = {
    "smartgrid-1": (3, 12),
    "smartgrid-2": (9, 12),
    "prob-termination-5": (6, 10),
    "prob-termination-7": (8, 10),
    "prob-termination-9": (10, 10),
    "prob-termination-12": (13, 20),
    "smartgrid-3": (27, 12),
    "reservoir-6": (64, 8),
    "window-20": (20, 20),
    "window-24": (24, 24),
    "window-28": (28, 28),
    "window-32": (32, 32),
    "smartgrid-4": (81, 12),
    "smartgrid-5": (243, 12),
    "reservoir-8": (256, 16),
    "random-walk-14": (500, 40),
    "random-walk-3": (500, 24),
    "random-walk-5": (500, 31),
    "random-walk-7": (500, 33),
    "reservoir-10": (1024, 32),
    "reservoir-4": (16, 4),
    "reservoir-12": (4096, 64),
}

method_rename = {
    "empirical": "Empirical",
    "miller": "Miller",
    "HyLeak": "HyLeak",
    "ChaoFON": "ChaoFON",
    "ChaoFRN": "ChaoFRN",
    "ChaoION": "ChaoSON",
    "ChaoIRN": "ChaoSRN",
    "ChaoFOM": "ChaoFOM",
    "ChaoFRM": "ChaoFRM",
    "ChaoIOM": "ChaoSOM",
    "ChaoIRM": "ChaoSRM",
}
method_order = [
    "Empirical",
    "Miller",
    "HyLeak",
    "ChaoFON",
    "ChaoFRN",
    "ChaoSON",
    "ChaoSRN",
    "ChaoFOM",
    "ChaoFRM",
    "ChaoSOM",
    "ChaoSRM",
]


In [14]:
df = pd.DataFrame(
    columns=["subject", "GT", "ratio", "method", "MI", "SE", "o.e."]
)
for subject in output_domain_size_dict.keys():
    data_path = f"../result/esti-merged-{subject}-i-xy.csv"
    if os.path.exists(data_path) == False:
        continue
    data = pd.read_csv(data_path)
    GT = data["GT"].values[0]
    data = data.drop(columns=["GT"])
    data_melt = data.melt(
        id_vars=["Nx", "trial"], var_name="method", value_name="MI"
    )
    # method rename
    data_melt["method"] = data_melt["method"].map(method_rename)
    data_melt["method"] = pd.Categorical(data_melt["method"], method_order)
    data_melt["SE"] = (data_melt["MI"] - GT) ** 2
    data_mean = (
        data_melt.drop(columns=["SE"])
        .groupby(["Nx", "method"], observed=False)
        .mean()
        .reset_index()
        .drop(columns=["trial"])
    )
    data_mse = (
        data_melt.drop(columns=["MI"])
        .groupby(["Nx", "method"], observed=False)
        .mean()
        .reset_index()
        .drop(columns=["trial"])
    )
    data_mean = data_mean.merge(data_mse, on=["Nx", "method"])
    NS_x, NS_y = output_domain_size_dict[subject]
    data_mean["ratio"] = data_mean["Nx"] / NS_y
    data_mean.loc[data_mean["ratio"] < 1, "ratio"] = 0.5
    data_mean = data_mean.drop(columns=["Nx"])
    data_mean["subject"] = subject
    data_mean["GT"] = GT
    data_mean["o.e."] = data_mean["MI"] - GT > 0
    df = pd.concat([df, data_mean], ignore_index=True)
df = df.dropna(subset=["MI"])
display(df)


/tmp/ipykernel_92898/829916277.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, data_mean], ignore_index=True)
/tmp/ipykernel_92898/829916277.py:40: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  df = pd.concat([df, data_mean], ignore_index=True)


,subject,GT,ratio,method,MI,SE,o.e.
0,smartgrid-1,0.065825,0.5,Empirical,0.394855,0.127088,True
1,smartgrid-1,0.065825,0.5,Miller,0.198559,0.030786,True
2,smartgrid-1,0.065825,0.5,HyLeak,0.113302,0.013150,True
3,smartgrid-1,0.065825,0.5,ChaoFON,0.386327,0.127033,True
4,smartgrid-1,0.065825,0.5,ChaoFRN,0.395682,0.128371,True
...,...,...,...,...,...,...,...
963,reservoir-12,1.604873,5.0,ChaoSRN,1.634559,0.000882,True
964,reservoir-12,1.604873,5.0,ChaoFOM,1.775405,0.029084,True
965,reservoir-12,1.604873,5.0,ChaoFRM,1.777474,0.029794,True
966,reservoir-12,1.604873,5.0,ChaoSOM,1.634573,0.000883,True


In [15]:
def compare(method1: str, method2: str, metric: str, ratio=None, debug=False):
    sub_df = df.copy().reset_index(drop=True)
    if ratio is not None:
        sub_df = sub_df[sub_df["ratio"].isin(ratio)]
    method1_df = sub_df[sub_df["method"] == method1].set_index(
        ["subject", "ratio"]
    )
    if method2 is not None:
        method2_df = sub_df[sub_df["method"] == method2].set_index(
            ["subject", "ratio"]
        )
    if debug and method2 is not None:
        pd.set_option("display.max_rows", None)
        display(method1_df[metric] - method2_df[metric])
        pd.reset_option("display.max_rows")
    if method2 is not None:
        diff = (method1_df[metric] - method2_df[metric]) > 0
        zero = (method1_df[metric] - method2_df[metric]) == 0
    else:
        diff = method1_df[metric] > 0
        zero = method1_df[metric] == 0
    return (diff.sum(), zero.sum(), len(diff), diff.sum() / len(diff))


def diff(method1: str, method2: str, metric: str, ratio=None, debug=False):
    sub_df = df.copy().reset_index(drop=True)
    if ratio is not None:
        sub_df = sub_df[sub_df["ratio"].isin(ratio)]
    method1_df = sub_df[sub_df["method"] == method1].set_index(
        ["subject", "ratio"]
    )
    if method2 is not None:
        method2_df = sub_df[sub_df["method"] == method2].set_index(
            ["subject", "ratio"]
        )
    if debug and method2 is not None:
        pd.set_option("display.max_rows", None)
        display(method1_df[metric] - method2_df[metric])
        pd.reset_option("display.max_rows")
    if method2 is not None:
        diff = (method1_df[metric] - method2_df[metric])
    else:
        diff = method1_df[metric]
    return diff.mean(), diff.std()


def relative_error(method, ratio=None, only_negative=False):
    sub_df = df.copy().reset_index(drop=True)
    if ratio is not None:
        sub_df = sub_df[sub_df["ratio"].isin(ratio)]
    method_df = sub_df[sub_df["method"] == method].set_index(
        ["subject", "ratio"]
    )
    return (
        (method_df["MI"] - method_df["GT"])
        * ((1 - method_df["o.e."]) if only_negative else 1)
        / method_df["GT"]
    ).mean()

# Empirical vs Chao

In [16]:
print("Empirical > ChaoFON:", compare("Empirical", "ChaoFON", "MI"))
print("Empirical > ChaoFRN:", compare("Empirical", "ChaoFRN", "MI"))
print("Empirical > ChaoSON:", compare("Empirical", "ChaoSON", "MI"))
print("Empirical > ChaoSRN:", compare("Empirical", "ChaoSRN", "MI"))

Empirical > ChaoFON: (32, 1, 88, 0.36363636363636365)
Empirical > ChaoFRN: (56, 1, 88, 0.6363636363636364)
Empirical > ChaoSON: (68, 1, 88, 0.7727272727272727)
Empirical > ChaoSRN: (87, 1, 88, 0.9886363636363636)


In [17]:
print("Empirical > ChaoFON:", compare("Empirical", "ChaoFON", "SE"))
print("Empirical > ChaoFRN:", compare("Empirical", "ChaoFRN", "SE"))
print("Empirical > ChaoSON:", compare("Empirical", "ChaoSON", "SE"))
print("Empirical > ChaoSRN:", compare("Empirical", "ChaoSRN", "SE"))

Empirical > ChaoFON: (30, 0, 88, 0.3409090909090909)
Empirical > ChaoFRN: (56, 0, 88, 0.6363636363636364)
Empirical > ChaoSON: (69, 0, 88, 0.7840909090909091)
Empirical > ChaoSRN: (88, 0, 88, 1.0)


# Flatten vs ByInput

In [18]:
print("ChaoFON > ChaoSON:", compare("ChaoFON", "ChaoSON", "MI"))
print("ChaoFRN > ChaoSRN:", compare("ChaoFRN", "ChaoSRN", "MI"))

ChaoFON > ChaoSON: (64, 0, 88, 0.7272727272727273)
ChaoFRN > ChaoSRN: (47, 0, 88, 0.5340909090909091)


In [19]:
print("ChaoFON > ChaoSON:", compare("ChaoFON", "ChaoSON", "SE"))
print("ChaoFRN > ChaoSRN:", compare("ChaoFRN", "ChaoSRN", "SE"))

ChaoFON > ChaoSON: (65, 0, 88, 0.7386363636363636)
ChaoFRN > ChaoSRN: (50, 0, 88, 0.5681818181818182)


# Refinement

In [20]:
print("ChaoFON > ChaoFRN:", compare("ChaoFON", "ChaoFRN", "MI"))
print("ChaoSON > ChaoSRN:", compare("ChaoSON", "ChaoSRN", "MI"))

ChaoFON > ChaoFRN: (56, 1, 88, 0.6363636363636364)
ChaoSON > ChaoSRN: (63, 2, 88, 0.7159090909090909)


In [21]:
print("ChaoFON > ChaoFRN:", compare("ChaoFON", "ChaoFRN", "SE"))
print("ChaoSON > ChaoSRN:", compare("ChaoSON", "ChaoSRN", "SE"))

ChaoFON > ChaoFRN: (57, 1, 88, 0.6477272727272727)
ChaoSON > ChaoSRN: (63, 2, 88, 0.7159090909090909)


# Müller's bias correction

In [22]:
print("Empirical > Miller:", compare("Empirical", "Miller", "SE"))
print("ChaoFRN > ChaoFRM:", compare("ChaoFRN", "ChaoFRM", "SE"))
print("ChaoSRN > ChaoSRM:", compare("ChaoSRN", "ChaoSRM", "SE"))

Empirical > Miller: (72, 0, 88, 0.8181818181818182)
ChaoFRN > ChaoFRM: (50, 0, 88, 0.5681818181818182)
ChaoSRN > ChaoSRM: (79, 0, 88, 0.8977272727272727)


In [23]:
print(
    compare("Empirical", None, "o.e.")[0] - compare("Miller", None, "o.e.")[0]
)
print(compare("ChaoFRN", None, "o.e.")[0] - compare("ChaoFRM", None, "o.e.")[0])
print(compare("ChaoSRN", None, "o.e.")[0] - compare("ChaoSRM", None, "o.e.")[0])


50
48
7


How much is the difference

In [24]:
print("Empirical > Miller:", diff("Empirical", "Miller", "MI"))
print("ChaoFRN > ChaoFRM:", diff("ChaoFRN", "ChaoFRM", "MI"))
print("ChaoSRN > ChaoSRM:", diff("ChaoSRN", "ChaoSRM", "MI"))

Empirical > Miller: (0.3539272669363526, 0.283879883525403)
ChaoFRN > ChaoFRM: (0.22483404995742973, 0.22199499734574057)
ChaoSRN > ChaoSRM: (0.05648676474732788, 0.08446017118961785)
